In [1]:
import pandas as pd
import datetime

In [21]:
df_sii = pd.read_excel('sii_2019.xlsx')
#df_exentas = pd.read_excel('facturas exentas 06.09.2022.xlsx')
df_acepta = pd.read_excel('acepta_2019.xlsx')
df_presupuesto = pd.read_excel('turbo_2019.xlsx')
df_sci = pd.read_excel('sci_2019.xlsx')

In [22]:
df_presupuesto.rename(columns = {'Rut': 'RUT Emisor', 'Folio': 'Folio_abreviado', 'NºDoc.': 'Folio'}, inplace = True)
df_sci.rename(columns = {'Rut Proveedor': 'RUT Emisor', 'Numero Documento': 'Folio'}, inplace = True)

In [23]:
todas_las_df = {'SII': df_sii, 'PRESUPUESTO': df_presupuesto, 'SCI': df_sci}
for nombre_tabla, df in todas_las_df.items():
    df.columns = df.columns + f' {nombre_tabla}'
    df['llave_id'] = df[f'RUT Emisor {nombre_tabla}'].astype(str) + df[f'Folio {nombre_tabla}'].astype(str)
    df.set_index('llave_id', drop = True, inplace = True)

In [26]:
# df_sii_exentas = pd.merge(df_sii, df_exentas, how = 'left', left_index = True, right_index = True)
df_sii_presupuesto = pd.merge(df_sii, df_presupuesto, how = 'left', left_index = True, right_index = True)
df_sii_presupuesto_sci = pd.merge(df_sii_presupuesto, df_sci, how = 'left', left_index = True, right_index = True)
df_sii_presupuesto_sci = df_sii_presupuesto_sci.drop_duplicates()

df_sii_presupuesto_sci['tiempo_diferencia'] = pd.to_datetime('today') - df_sii_presupuesto_sci['Fecha Docto. SII']
df_sii_presupuesto_sci['esta_al_dia'] = df_sii_presupuesto_sci['tiempo_diferencia'] <= datetime.timedelta(8)

df_sii_presupuesto_sci.to_excel('PLANILLA_MATRIZ.xlsx')


In [ ]:
df_sii_presupuesto_sci